# Search Dynamics: Visualizing Population Evolution

This notebook visualizes *how* NSGA-II converges to the Pareto front over time.

## Goals
1.  **Pareto Front Evolution**: Watch the population converge generation by generation.
2.  **Diversity Over Time**: Track how spread-out the solutions are.
3.  **Crowding Distance**: Understand the final population distribution.
4.  **Export Animation**: Save the evolution as a GIF for presentations.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.colors import Normalize
from matplotlib.cm import ScalarMappable
from matplotlib.animation import FuncAnimation, PillowWriter
from IPython.display import Image, display

# VAMOS imports
from vamos import optimize
from vamos.algorithms import NSGAIIConfig
from vamos.problems import ZDT1
from vamos.foundation.metrics.hypervolume import compute_hypervolume

## 1. Collect Population Snapshots
We run NSGA-II with increasing budgets to capture the Pareto front at different stages.

In [ ]:
def collect_snapshots(problem, budgets, pop_size=56, seed=42):
    """
    Run NSGA-II with increasing budgets and collect objective values.
    Returns a dict: {budget: F_array}
    """
    snapshots = {}

    for budget in budgets:
        cfg = (
            NSGAIIConfig.builder()
            .pop_size(pop_size)
            .offspring_size(14)
            .crossover("blx_alpha", prob=0.88, alpha=0.94, repair="clip")
            .mutation("non_uniform", prob="0.45/n", perturbation=0.3)
            .selection("tournament", pressure=9)
            .repair("round")
            .archive(size=56).archive_type("hypervolume")
            
            .build()
        )
        res = optimize(**dict(problem=problem, algorithm="nsgaii", algorithm_config=cfg, termination=("n_eval", budget), seed=seed))
        if res.F is not None:
            snapshots[budget] = res.F.copy()
        else:
            snapshots[budget] = np.array([])

    return snapshots


# Run on ZDT1
problem = ZDT1(n_var=30)
budgets = [50, 100, 200, 500, 1000, 2000, 5000]

print("Collecting population snapshots...")
snapshots = collect_snapshots(problem, budgets)
print(f"Captured {len(snapshots)} snapshots.")

## 2. Pareto Front Evolution
Visualize how the population moves towards the Pareto front.

In [ ]:
from vamos.foundation.metrics.pareto import pareto_filter

# True Pareto front for ZDT1
f1_true = np.linspace(0, 1, 100)
f2_true = 1 - np.sqrt(f1_true)

fig, axes = plt.subplots(2, 4, figsize=(16, 8))
axes = axes.flatten()

for i, budget in enumerate(budgets):
    ax = axes[i]
    F = snapshots[budget]

    # Plot true front
    ax.plot(f1_true, f2_true, "k--", alpha=0.5, label="True PF")

    # Plot population
    if len(F) > 0:
        F_plot = pareto_filter(F)
        if F_plot is not None and F_plot.size:
            ax.scatter(F_plot[:, 0], F_plot[:, 1], c="blue", alpha=0.6, s=20)

    ax.set_title(f"Evals: {budget}")
    ax.set_xlabel("$f_1$")
    ax.set_ylabel("$f_2$")
    ax.set_xlim(-0.1, 1.2)
    ax.set_ylim(-0.1, 1.5)
    ax.grid(True, alpha=0.3)

# Hide unused subplot
if len(budgets) < len(axes):
    for j in range(len(budgets), len(axes)):
        axes[j].axis("off")

plt.suptitle("Pareto Front Evolution (ZDT1 + NSGA-II)", fontsize=14)
plt.tight_layout()
plt.show()

## 3. Diversity Over Time
We measure diversity as the standard deviation of $f_1$ values (spread across the front).

In [ ]:
def compute_diversity(F):
    """Compute diversity as std of f1 (spread across front)."""
    if F is None or len(F) == 0:
        return 0.0
    return float(np.std(F[:, 0]))


def compute_spread(F):
    """Compute spread as range of f1."""
    if F is None or len(F) == 0:
        return 0.0
    return float(np.max(F[:, 0]) - np.min(F[:, 0]))


diversity_data = []
ref_point = np.array([1.1, 1.1])

for budget in budgets:
    F = snapshots[budget]
    div = compute_diversity(F)
    spread = compute_spread(F)
    hv = compute_hypervolume(F, ref_point) if len(F) > 0 else 0.0
    diversity_data.append({"Budget": budget, "Diversity": div, "Spread": spread, "HV": hv})

df_div = pd.DataFrame(diversity_data)
df_div

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

# HV over time
axes[0].plot(df_div["Budget"], df_div["HV"], "o-", color="green")
axes[0].set_xlabel("Evaluations")
axes[0].set_ylabel("Hypervolume")
axes[0].set_title("Hypervolume Over Time")
axes[0].grid(True, alpha=0.3)

# Diversity over time
axes[1].plot(df_div["Budget"], df_div["Diversity"], "o-", color="blue")
axes[1].set_xlabel("Evaluations")
axes[1].set_ylabel("Diversity (Std $f_1$)")
axes[1].set_title("Diversity Over Time")
axes[1].grid(True, alpha=0.3)

# Spread over time
axes[2].plot(df_div["Budget"], df_div["Spread"], "o-", color="orange")
axes[2].set_xlabel("Evaluations")
axes[2].set_ylabel("Spread (Range $f_1$)")
axes[2].set_title("Spread Over Time")
axes[2].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 4. Crowding Distance Histogram (Final Population)
Crowding distance measures how "crowded" each solution is. Higher is better (more unique).

In [ ]:
def compute_crowding_distance(F):
    """
    Compute crowding distance for a 2D front.
    """
    n = len(F)
    if n == 0:
        return np.array([])

    cd = np.zeros(n)

    for m in range(F.shape[1]):
        sorted_idx = np.argsort(F[:, m])
        cd[sorted_idx[0]] = np.inf
        cd[sorted_idx[-1]] = np.inf

        f_range = F[sorted_idx[-1], m] - F[sorted_idx[0], m]
        if f_range == 0:
            continue

        for i in range(1, n - 1):
            cd[sorted_idx[i]] += (F[sorted_idx[i + 1], m] - F[sorted_idx[i - 1], m]) / f_range

    return cd


# Final front
F_final = snapshots[max(budgets)]
cd = compute_crowding_distance(F_final)

# Replace inf with max finite for visualization
cd_viz = cd.copy()
cd_viz[np.isinf(cd_viz)] = np.max(cd_viz[np.isfinite(cd_viz)]) * 1.2

plt.figure(figsize=(10, 5))
plt.hist(cd_viz, bins=20, edgecolor="black", alpha=0.7)
plt.xlabel("Crowding Distance")
plt.ylabel("Count")
plt.title("Crowding Distance Distribution (Final Population)")
plt.grid(True, alpha=0.3)
plt.show()

## 5. Export Animation as GIF
Create an animated GIF showing the Pareto front evolution (great for presentations).

In [ ]:
from vamos.foundation.metrics.pareto import pareto_filter


def create_evolution_gif(snapshots, budgets, output_path="pareto_evolution.gif"):
    """
    Create an animated GIF showing Pareto front evolution.
    """
    fig, ax = plt.subplots(figsize=(8, 6))

    def animate(frame_idx):
        ax.clear()
        ax.plot(f1_true, f2_true, "k--", alpha=0.5, label="True PF")

        budget = budgets[frame_idx]
        F = snapshots[budget]

        if len(F) > 0:
            F_plot = pareto_filter(F)
            if F_plot is not None and F_plot.size:
                ax.scatter(F_plot[:, 0], F_plot[:, 1], c="blue", alpha=0.6, s=30)

        ax.set_title(f"Pareto Front Evolution - Evals: {budget}")
        ax.set_xlabel("$f_1$")
        ax.set_ylabel("$f_2$")
        ax.set_xlim(-0.1, 1.2)
        ax.set_ylim(-0.1, 1.5)
        ax.grid(True, alpha=0.3)
        return []

    anim = FuncAnimation(fig, animate, frames=len(budgets), interval=500, blit=True)

    # Save as GIF
    writer = PillowWriter(fps=2)
    anim.save(output_path, writer=writer)
    plt.close(fig)
    print(f"GIF saved to: {output_path}")
    return output_path


# Create and display the GIF
gif_path = create_evolution_gif(snapshots, budgets)
display(Image(filename=gif_path))